# Shapley Values

머신러닝 모형은 예측 성능을 높이기 위해 점점 복잡해지고 있다. 하지만 예측의 정확도는 증가하고 있으나, 모형의 복잡도는 설명 가능성 또는 해석 가능성은 점차적으로 낮아지고 있다. 여기서 설명 가능이란 말은 **모형의 예측과정을 사람이 쉽게 이해하는 정도**로 정의할 수 있다. 예를 들어 3개의 설명변수가 있는 선형모형의 예측치는
$$
f(x)=\beta_0 + \beta_1x_1 + \beta_2x_2+\beta_3x_3
$$
이다. 예측치 $f(x)$에 대한 설명변수 $x_i$의 기여도(contribution)는 $\beta_ix_i$가 되는 것을 알 수 있다. 이처럼 선형모형에서는 설명변수와 예측값의 관계를 쉽게 도출할 수 있지만, 대부분의 머신러닝에서는 설명변수가 내부적으로 어떻게 연결되어 예측치를 만들어 내는지를 모르는 black-box 모형이다.

선형모형에서 설명변수 $x_2$를 제거하면 $\beta_2x_2$만큼 예측값이 줄어드는 것처럼, 머신러닝 모형의 설명 가능성에 대한 핵심적인 질문은 **특정 설명변수를 모델에서 제거하면 예측치는 얼마나 변화하는가?** 이다. 이러한 아이디어에서 나온 것이 **SHAP(Shapley Additive Explanation) value**이며 SHAP은 각 설명변수가 예측치에 미치는 영향을 수량화한 것이라고 할 수 있다.

SHAP은 개별 예측치에 대한 모형 무관 설명도구(Individualized model-agnostic explainer)로서, 경제학자인 Lloyd Shapley가 제안한 게임이론을 응용한 explainer이다. 모형 무관의 의미는 딥러닝 모형과 같이 내부적으로 어떻게 작동하는지 알지 못하더라도 입력과 출력만 관측되면 적용이 가능하다는 의미이다.

---

## Shapley value의 기본 개념

어떤 [협력게임](https://ko.wikipedia.org/wiki/%ED%98%91%EB%A0%A5_%EA%B2%8C%EC%9E%84)에서 $M$명의 선수가 있다고 하자. $F=\left\{ 1,2,...,M \right\}$으로 선수들의 집합을 정의할 때, $S$가 $F$의 부분집합이면 이를 연대(coalition)라고 한다. 예를 들어, 3명의 선수가 있을 때
$$
F=\left\{ 1,2,3 \right\} and\ S \in \left\{ \emptyset,\left\{ 1 \right\},\left\{ 2 \right\},\left\{ 3 \right\}\left\{ 1,2 \right\}\left\{ 1,3 \right\}\left\{ 2,3 \right\}\left\{ 1,2,3 \right\} \right\}
$$
이 된다. $F$도 하나의 연대이고 이를 전체연대(grand coalition)라고 한다. $v(S)$를 연대 $S$의 가치라고 할 때, 이 $v(s)$를 이용하여 Shapley value를 구하는 과정을 살펴본다.

Shapley value에서 선수가 없는 게임의 가치를 $v(\emptyset)=0$으로 가정한다. 세 명의 선수 중 선수2와 선수3은 기술적으로 거의 같다고 했을 때,
$$ 
v(\left\{ 1,2 \right\})-v(\left\{ 1 \right\})= 10000
$$
로, 선수2의 기여도는 10000이지만
$$
v(\left\{ 1,2,3 \right\})-v(\left\{ 1,2 \right\})= 1000
$$
이면 선수3의 기여도 1000으로 평가된다. 선수2와 선수3의 기량이 비슷하기 때문에 충분히 일어날 수 있는 일이다. 만약 선수3이 면저 게임에 참여했으면
$$
v(\left\{ 1,3 \right\})-v(\left\{ 1 \right\}) \approx 10000
$$
이 되어서 선수3의 기여도가 10000이 될 수 있었을 것이다. 즉, 선수의 기여도가 게임에 참석하는 순서에 영향을 받을 수 밖에 없게 된다.

선수 $i$의 순서에 영향을 받지 않는 기여도를 구하기 위해 $F$의 모든 순열(permutation)에서 선수 $i$의 기여도를 모두 구한 후 평균을 산출한다.

<center>

| permutation | 선수1 | 선수2 | 선수3 |
|:-----------:|:------:|:------:|:------:|
| {1,2,3}    | $ v(\{1\}) - v(\emptyset) $ | $ v(\{1,2\}) - v(\{1\}) $ | $ v(\{1,2,3\}) - v(\{1,2\}) $ |
| {1,3,2}    | $ v(\{1\}) - v(\emptyset) $ | $ v(\{1,3\}) - v(\{1\}) $ | $ v(\{1,2,3\}) - v(\{1,3\}) $ |
| {2,1,3}    | $ v(\{1,2\}) - v(\{2\}) $ | $ v(\{2\}) - v(\emptyset) $ | $ v(\{1,2,3\}) - v(\{1,2\}) $ |
| {2,3,1}    | $ v(\{1,2,3\}) - v(\{2,3\}) $ | $ v(\{2,3\}) - v(\{2\}) $ | $ v(\{2\}) - v(\emptyset) $ |
| {3,1,2}    | $ v(\{1,3\}) - v(\{3\}) $ | $ v(\{1,2,3\}) - v(\{1,3\}) $ | $ v(\{3\}) - v(\emptyset) $ |
| {3,2,1}    | $ v(\{1,2,3\}) - v(\{2,3\}) $ | $ v(\{2,3\}) - v(\{3\}) $ | $ v(\{3\}) - v(\emptyset) $ |
| **평균**   | $ \phi_1 $ | $ \phi_2 $ | $ \phi_3 $ |

</center>

위의 표에서 permutation은 게임에 참가하는 순서를 나타내고 이 순서에 따라 3명의 선수들의 기여도를 구한 후, 이를 평균하여 Shapley value $\phi_1,\phi_2,\phi_3$으로 표기하고 있다. 표에서 볼 수 있듯이 선수의 기여도는 게임에 참여하는 순서에 의존하지만, 연대의 가치 $v(S)$는 순서에 의존하지 않는다. 예를 들어,
$$
v(\{1,2\})=v(\{2,1\})\ and\ v(\{1,2,3\})=v(\{2,1,3\})
$$
을 가정한다. 또한
$$
\phi_1+\phi_2+\phi_3=v(\{1,2,3\})=v(F)
$$
를 만족한다는 것을 쉽게 확인할 수 있다. 즉, 가치함수 $v(S)$에 어떤 값을 부여해도 $\phi_1+\phi_2+\phi_3=v(F)$를 만족하는 것을 알 수 있으며 이를 Shapley value의 덧샘법칙(additivity)라고 한다.  
예를 들어, 임의로 $v(\{1\})=1$, $v(\{2\})=2$, $v(\{3\})=3$, $v(\{1,2\})=-2$, $v(\{1,3\})=5$, $v(\{2,3\})=3$, $v(\{1,2,3\})=7$로 부여하더라도 $\phi_1+\phi_2+\phi_3=7$이 된다.

$F=\{1,2,...,M\}$일 때 선수 $i$에 대한 Shapley value는 다음과 같이 정의된다.
$$
\phi_i = \frac{1}{\left| F \right|!}\sum_{p}^{}(v(S\cup i)-v(S)) \tag 1
$$

<center>

| Notation              |     설명     |
| :------:              | :---------: |
|    $p$                | permutation |
| $\| F \|$             | $F$의 크기 M |
| $S$                   | 고정된 permutation $p$에서 선수 $i$ 이전에 게임에 참가한 선수들의 연대 |

</center>  

예를 들어 permutation $p=\{1,3,2\}$ 일 때, $i=1$이면 $S=\{\emptyset\}$, $i=3$이면 $S=\{1\}$이 된다. 그러므로 permutation $p$가 변하면 $v(S \cup u)$의 $S$도 변하게 된다. 위의 표에서 $\phi_1$을 계산하는 열을 살펴보면 식 $(1)$을 쉽게 이해할 수 있다. 해당 식에서 $S$는 자체의 permutation과 $S \cup i$다음에 나타나는 $(\left| F \right| - \left| S \right|-1)$ 개의 나머지 선수들의 permutation에 대해서도 동일한 $v(S \cup i)-v(S)$ 값을 가지므로 식 $(1)$을 다음과 같이 재표현할 수도 있다.


$$
\phi_i = \sum_{S\subseteq F-\left\{ i \right\}}^{}\frac{\left| S \right|!(\left| F \right|-\left| S \right|-1)!}{\left| F \right|}(v(S\cup i)-v(S)) \tag2
$$

Shapley value의 덧셈법칙(additivity)에 의해 $\sum_{i=1}^{M}\phi_i=v(F)$를 만족한다.

Shapley value의 추가적인 성질으로, 모든 연대(coalition) $S$에 대해 $v(S \cup i) = v(S)$이면 $\phi_i=0$이고 $v(S\cup i)= v(S \cup j)$이면 $\phi_i=\phi_j$임을 식$(1)$을 통해 알 수 있다. 전자를 **dummy**라고 말하며 후자를 **symmetry**라고 한다. dummy는 선수 $i$가 모든 연대에 대해 추가적인 기여도가 없다면 선수 $i$의 기여도는 0이라는 의미이고 symmetry는 두 명의 선수가 모든 연대에 대해 추가적인 기여도가 동일하다면 두 선수의 기여도는 동일하나든 의미를 가진다.

---

## 머신러닝에서의 Shapley value

앞에서 설명한 Shapley value의 개념을 머신러닝 모형 $f(x)$에 적용하기 위해 앞의 예시에서 언급한 '선수'를 정해야 한다. 설명변수 $x=(x_1,x_2,...,x_M)$으로 정의할 때, 머신러닝 모형의 예측값 $f(x)$ 값에 기여하는 것은 설명변수이므로 $x_1,x_2,...,x_M$이 선수가 된다.  
Shapley value의 가치함수를 정의하기 위해
$$
f(\phi)=E(f(x)) \tag 3
$$
로 정의하고
$$
v(S)=f(S)-E(f(x)) \tag 4
$$
로 정의한다. 여기서 $F=\{x_1,x_2,...,x_M\}$이고 $S$는 $F$의 부분집합이다. $f(x)$는 $x=(x_1,x_2,...,x_M)$가 관측되었을 때의 모델 $f$의 예측값이며, $E(f(x))$는 예측값의 기댓값이므로 학습데이터로부터 임의로 뽑은 $k$의 표본을 이용하여
$$
\frac{1}{k}\sum_{i=1}^{k}f(x_i) \tag 5
$$
로 추정한다. 식 $(3)$과 $(4)$를 통해
$$
v(\phi) = f(\phi) - E(f(x))=0
$$
이 되어 Shapley value에서 정의한 가치함수 $v$의 기본요건을 충족하는 것을 알 수 있다.

식 $(4)$의 $v(S)$를 계산하기 위해서는 $f(S)$를 계산하여야 한다. 예를 들어, $F=\{x_1,x_2,...,x_5\}$일 때 연대(coalition) $S=\{x_1,x_3\}$이면 $f(x_1,x_3)$를 계산하여야 한다. $f(x_1,x_3)$의 계산은 두 가지 방법이 있다.

- 방법 1 : 원래 모형 $f(x_1,x_2,x_3,x_4,x_5)$와 동일한 모형 $f(x_1)$을 재추정  
- 방법 2 : 이미 추정된 원래 모형 $f(x_1,x_2,x_3,x_4,x_5)$으로부터 $f(x_1,x_3)$을 계산

방법 1의 경우, 설명변수가 적고 선형회귀모델과 같이 간단하 모형 이외에는 계산 부담이 너무 크므로 후자의 방법으로 식 $(4)$의 $f(S)$를 계산할 수 밖에 없다.

***Lundberg & LEE***는
$$
f(S) \approx E(f(x)|S)
$$
로 근사시키는 방법을 제시했다. 설명변수의 부분집합 $S$가 주어진 상태에서의 기대치는 $f(S)$와 일반적으로 일치하지는 않지만 오직 부분집합 $S$만의 함수이고 $E(f(x)|F)=E(f(x)|x)=f(x)$가 된다. 그러므로 $E(f(x)|S)$를 이용하여 다음과 같이 정의한 Shapley value는 앞에서 언급한 덧샘법칙(additivity)를 만족하게 된다.

$v(F)=f(x)-E(f(x))$이므로 Shapley의 덧셈법칙에 의해 $\sum_{i=1}^{M}\phi_i=f(x)-E(f(x))$이 된다. 즉, $f(x)=\sum_{i=1}^{M}\phi_i+E(f(x))$가 된다. 이러한 $\phi_i$를 SHAP value라고 ***Lundberg & Lee***는 정의하였다.

$\bar{S}=F-S$라고 정의하면
$$
E(f(F)|S)=\int_{}^{}f(S,\bar{S})p(S,\bar{S}|S)d\bar{S}
$$
가 된다. $\bar{S}$에 속하는 설명변수를 적분하므로 $E(f(F)|S)$는 오직 $S$에 속하는 설명변수의 함수가 되게 한다. 그러나 조건부 분포함수 $p(S,\bar{S}|S)$는 대부분의 경우 계산이 불가능하거나 어렵기 때문에 SHAP에서는 $p(S,\bar{S}|S)=p(S,\bar{S})$, 즉 설명변수가 서로 독립이라는 가정 하에서
$$
E(f(F)|S)=\int_{}^{}f(S,\bar{S})p(S,\bar{S})d\bar{S} \tag 6
$$
으로 정의하게 된다. 이는 다음과 같이 추정할 수 있다.
$$
\frac{1}{k}\sum_{i=1}^{k}f(S,\bar{S}^{(i)})
$$
여기서 $k$는 학습데이터로부터 임의로 추출된 표본 수이다. 예를 들어 $F=\{x_1,x_2,...,x_5\}$, $\bar{S}=\{x_2,x_4,x_5\}$이면
$$
\frac{1}{k}\sum_{i=1}^{k}f(x_1,x_2^{(i)},x_3,x_4^{(i)},x_5^{(i)}) \tag 7
$$
로 추정하게 된다.

식 $(5)$와 $(7)$을 구하기 위해 $E(f(x))$와 SHAP value의 추정치를 구체적으로 구하기 위해선 다음과 같이 shap을 설치해야한다.

```
pip install shap
```

shap 라이브러리는 다음과 같이 매우 간단한 구조를 가지고 있다.

```python
import shap
explainer = shap.Explainer(predict_fn, data1)
shap_values = explainer(data2)
```

먼저 shap을 호출한 후, `shap.Explainer` 내에 옵션으로 미리 학습된 모델의 예측함수와 shap value를 추정할 데이터 `data1`을 지정하여 `shap.Explainer`를 동기화한다. shap value를 구할 표본(들)인 `data2`를 explainer에 적용하여 `shap_values`로 저장한다. shap 라이브러리는 `Explainer` 이외에도 `KernelExplainer`,`TreeExplainer`,`DeepExplainer` 등의 클래스도 있으며  문법은 앞의 설명과 거의 동일하다.

다음은 Boston housing 데이터를 이용하여 주택가격을 예측하기 위해 선형회귀 모형을 적합하고 있다.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [5]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
boston_df = pd.read_csv(url)

X=boston_df[['lstat','age','rad','nox']]
y=pd.Series(boston_df['medv'])

X100=X[100:200]

In [8]:
model = LinearRegression()
model.fit(X,y);

`X100=X[100:200]`으로 정의하여 해당 데이터를 이용하여 다음과 같이 식 $(5)$와 $(8)$을 계산한다. 그러므로 $k=100$이 된다. 아래 코드에서 `shap.Explainer`에 적용할 머신러닝 모형과 데이터(X100)를 입력한 후, 첫 번째 표본의 SHAP value를 구한다.

In [12]:
print(model.predict(X.iloc[0:1]))

import shap
explainer = shap.Explainer(model,X100)

shap_value = explainer(X.iloc[0:1])
print(shap_value)

[30.44394072]
.values =
array([[ 7.80921425, -0.73084402,  0.12905011,  0.23758952]])

.base_values =
array([22.99893087])

.data =
array([[ 4.98 , 65.2  ,  1.   ,  0.538]])


위의 결과에서 `.value`는 식 $(7)$으로부터 계산한 4개 설명변수의 SHAP value를 보여준다. `.base`는 식 $(5)$로부터 구한 예측 기댓값이며, `.data`는 첫 번째 표본의 설명변수 값이다. `shap.Explainer`를 이용하여 SHAP value를 계산하기 위해 $O(2^{M-1}Mk)$의 비용이 필요하다. 여기서 $M$은 설명변수의 수, $k$는 식 $(7)$을 계산하기 위한 표본 수를 의미한다. 그러므로 설명변수의 수가 조금만 커지더라도 계산 부담이 기하급수적으로 증가하게 된다. 이러한 문제를 해결하기 위해 제안된 방법이 kernelSHAP이다.

---

## Kernel SHAP